<a href="https://colab.research.google.com/github/zihyang-227/4120finalproject/blob/main/5_Module_2_Data_Modeling_and_Decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture Module 2: Logical Design: Conceptual Data Representation

## LinkedIn Social Analysis

Our second module explores concepts in:

* Designing data representations to capture important relationships
* Reasoning over graphs
* Exploring and traversing graphs


Subsequently, in the next module, we'll look at how *physical design* (indexing, data layout) and *algorithms* can affect performance.

## Generality of Data Models

We have claimed that data can be represented as a tree, as tables, or as graphs -- and all are equivalent. We'll see this in action here.

## Hierarchical data



### Preliminaries

We'll use MongoDB on the cloud as a sample NoSQL database.

We'll first collect Colab's host IP address, which you might need if you aren't able to connect to the database.  If you get an authorization error in connecting to MongoDB, you'll need to post this IP address to Ed Discussion so we can add permissions to make a request.

In [1]:
#TODO: fill in from Ed
%env PASSWORD=4rmWiAV8bTdwSWbL

env: PASSWORD=4rmWiAV8bTdwSWbL


In [2]:
!curl ipecho.net/plain

35.237.80.17

In [3]:
!pip3 install pymongo
!pip3 install lxml
!pip3 install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 20.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np

# JSON parsing
import json

# HTML parsing
from lxml import etree
import urllib

# DuckDB RDBMS
import duckdb

# Time conversions
import time

# NoSQL DB
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

## Our Example Dataset

A crawl of LinkedIn, stored as a sequence of JSON objects (one per line).  Here's a scan through the sample dataset, taken from Kaggle (https://www.kaggle.com/linkedindata/linkedin-crawled-profiles-dataset).  We have subsequently removed all names of individuals.

In [5]:
!wget -nc https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl

--2025-09-29 16:22:54--  https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.196.207, 173.194.217.207, 108.177.11.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.196.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179851696 (172M) [application/octet-stream]
Saving to: ‘linkedin_anon.jsonl’

linkedin_anon.jsonl 100%[===================>] 171.52M   118MB/s    in 1.4s    

2025-09-29 16:22:56 (118 MB/s) - ‘linkedin_anon.jsonl’ saved [179851696/179851696]



In [6]:
%%time
# 50K records from linkedin
linked_in = open('linkedin_anon.jsonl')

people = []

for line in linked_in:
    person = json.loads(line)
    people.append(person)

people_df = pd.DataFrame(people)
print ("%d records"%len(people_df))

people_df

50000 records
CPU times: user 4.69 s, sys: 673 ms, total: 5.36 s
Wall time: 7.88 s


,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,moist-vodka,"{'family_name': 'Post', 'given_name': 'Belvede...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,https://www.linkedin.com/in/moist-vodka,None,None,NaN,None,None,None,None,None
1,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
2,tart-acorn,"{'family_name': 'Hannay', 'given_name': 'Passe...","San Francisco, California","[DNA, Nanotechnology, Molecular Biology, Softw...",Research,I am interested in inventing new methods to co...,https://www.linkedin.com/in/tart-acorn,"[{'major': 'Biophysics', 'end': '2009', 'name'...",None,0.0,"[{'org': 'UCSF', 'title': 'Assistant Professor...",None,[{'from': 'Wyss Institute for Biologically Ins...,"personal genomics, nanotechnology",None
3,objective-riesling,"{'family_name': 'Carnegie', 'given_name': 'Pas...",San Francisco Bay Area,None,Information Technology and Services,OBJECTIVE<Primary> Work on an interesting and ...,https://www.linkedin.com/in/objective-riesling,None,"{'affilition': ['Big Data, Low Latency', 'Expe...",5.0,"[{'org': '<Online Recruiting Company>', 'desc'...",None,"[{'from': '<Employee Benefits, Administration ...",None,None
4,generative-amberjack,"{'family_name': 'Duncan', 'given_name': 'Merri...","Chennai Area, India","[Program Management, French, Avionics, Embedde...",Aviation & Aerospace,"Experience in Avionics Systems, Embedded Syste...",https://www.linkedin.com/in/generative-amberjack,"[{'start': '1988', 'end': '1989', 'name': 'Eco...",{'member': 'Member of Project Management Insti...,NaN,None,None,None,"Literature, Philosophy, Music",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,glowing-flush,"{'family_name': 'Kincaid', 'given_name': 'Pass...",Greater Chicago Area,"[Spanish-speaking, Cicerone-Certified Beer Ser...",Marketing and Advertising,Sales and marketing professional specializing ...,https://www.linkedin.com/in/glowing-flush,"[{'start': '2007', 'major': 'PR and Advertisin...",None,5.0,"[{'org': 'Louis Glunz Beer Inc.', 'title': 'On...",None,"[{'from': 'Peet's Coffee and Tea', 'to': 'Expl...","craft beer industry, coffee industry, running,...",None
49996,grouchy-flight,"{'family_name': 'Ogilvy', 'given_name': 'Bulli...",Greater Atlanta Area,None,Financial Services,Accomplished business development manager expe...,https://www.linkedin.com/in/grouchy-flight,"[{'major': 'Marketing Focus', 'end': '2008', '...","{'member': 'Sigma Chi Fraternity', 'affilition...",14.0,"[{'org': 'Georgia-Pacific LLC', 'title': 'Acco...",None,"[{'from': 'Bayer Advanced', 'to': 'BBDO', 'tit...",None,[National Deans List]
49997,dense-bell,"{'family_name': 'Macdougall', 'given_name': 'B...","Calgary, Canada Area","[Project Management, Electrical Engineering, M...",Design,Brad Gibson is a recognized expert in power qu...,https://www.linkedin.com/in/dense-bell,[{'major': 'Engineering Physics (Solid State E...,"{'member': 'IEEE, APEGGA, APEGBC, PEO, APEGS, ...",42.0,"[{'org': 'DIALOG', 'desc': 'Electrical Enginee...","Data center design, high reliability power, po...","[{'from': 'Current Thinking Inc.', 'to': 'The ...",None,None
49998,brave-hoops,"{'family_name': 'Forsyth', 'given_name': 'Cadb...",San Francisco Bay Area,"[Corporate Social Responsibility, Public Polic...",Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,"[{'major': 'Law', 'end':

## NoSQL storage

For this part we will give you read-only access to our copy of MongoDB.

We may need to tell MongoDB to add your Colab IP address (so you can talk to the machine).

In [7]:
import os
# Store in MongoDB and in an in-memory list

START = 0
# We already have the data loaded into MongoDB, so we won't actually
# read all 50000 records.  We'll test by reading + writing the first
# 3700 though!
LIMIT = 37#00

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pymongo.client_options
import pymongo
password = os.getenv('PASSWORD')

if password is None:
    raise Exception("You must set the PASSWORD environment variable")

uri = "mongodb+srv://cis5450:" + password + "@test2450.3emsbl6.mongodb.net/?retryWrites=true&w=majority&appName=Test2450"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

linkedin_db = client['linkedin']
linked_in = open('linkedin_anon.jsonl')

print('MongoDB has the following databases' + str(client.list_database_names()))

people = 0
for line in linked_in:
    person = json.loads(line)
    if people >= START:
        try:
            linkedin_db.posts.insert_one(person)
        except DuplicateKeyError as e:
            print (e)
            pass
        except OperationFailure as e:
            # If the above still uses our cluster, you'll get this error in
            # attempting to write to our MongoDB client because we haven't
            # given you write access
            if ("user is not allowed to do action [insert]" not in str(e)):
              print (e)
            pass
    people = people + 1
    if people >= LIMIT:
        break

MongoDB has the following databases['linkedin', 'admin', 'local']


In [8]:
# Build a list of the JSON elements
list_for_comparison = []

people = 0
for line in linked_in:
    person = json.loads(line)
    if people >= START:
        try:
            list_for_comparison.append(person)
        except DuplicateKeyError:
            pass
        except OperationFailure:
            # If the above still uses our cluster, you'll get this error in
            # attempting to write to our MongoDB client
            pass
    people = people + 1
    if people >= LIMIT:
        break

In [9]:
list_for_comparison

[{'_id': 'brutal-web',
  'name': {'family_name': 'Campbell', 'given_name': 'Poole'},
  'locality': 'Greater Los Angeles Area',
  'skills': ['Data Analysis',
   'Market Research',
   'Social Networking',
   'PowerPoint',
   'Social Media',
   'SPSS',
   'Advertising',
   'Marketing Research',
   'Public Relations',
   'Online Advertising',
   'Event Planning',
   'Analytics',
   'Microsoft Office',
   'Microsoft Excel',
   'Fundraising',
   'Marketing',
   'Social Media Marketing',
   'Product Development'],
  'industry': 'Market Research',
  'summary': None,
  'url': 'https://www.linkedin.com/in/brutal-web',
  'education': [{'start': '2005',
    'major': 'Psychology',
    'end': '2009',
    'name': 'University of Southern California',
    'degree': 'B.A.'}],
  'group': None,
  'interval': 17.0,
  'experience': [{'org': 'Ace Metrix',
    'desc': "I'm responsible for a multitude of tasks in 3 main areas; first, I manage all day to day operations collecting data. Second, I process all dat

In [10]:
# Two ways of looking up skills, one based on an in-memory
# list, one based on MongoDB queries

def find_skills_in_list(skill):
    for post in list_for_comparison:
        if 'skills' in post:
            skills = post['skills']
            if skills is not None:
              for this_skill in skills:
                  if this_skill == skill:
                      return post
    return None

def find_skills_in_mongodb(skill):
    return linkedin_db.posts.find_one({'skills': skill})

In [11]:
%%time
find_skills_in_list('Marketing')

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 23.6 µs


{'_id': 'brutal-web',
 'name': {'family_name': 'Campbell', 'given_name': 'Poole'},
 'locality': 'Greater Los Angeles Area',
 'skills': ['Data Analysis',
  'Market Research',
  'Social Networking',
  'PowerPoint',
  'Social Media',
  'SPSS',
  'Advertising',
  'Marketing Research',
  'Public Relations',
  'Online Advertising',
  'Event Planning',
  'Analytics',
  'Microsoft Office',
  'Microsoft Excel',
  'Fundraising',
  'Marketing',
  'Social Media Marketing',
  'Product Development'],
 'industry': 'Market Research',
 'summary': None,
 'url': 'https://www.linkedin.com/in/brutal-web',
 'education': [{'start': '2005',
   'major': 'Psychology',
   'end': '2009',
   'name': 'University of Southern California',
   'degree': 'B.A.'}],
 'group': None,
 'interval': 17.0,
 'experience': [{'org': 'Ace Metrix',
   'desc': "I'm responsible for a multitude of tasks in 3 main areas; first, I manage all day to day operations collecting data. Second, I process all data numbers for client deliverables

In [12]:
%%time
find_skills_in_mongodb('Marketing')

CPU times: user 2.18 ms, sys: 47 µs, total: 2.23 ms
Wall time: 22.9 ms


{'_id': 'parallel-functionality',
 'name': {'family_name': 'Howard', 'given_name': 'Jenkins'},
 'locality': 'Pompano Beach, Florida',
 'skills': ['Collateral',
  'Research',
  'Email Marketing',
  'Marketing Communications',
  'Direct Marketing',
  'Event Planning',
  'Marketing',
  'Social Media Marketing',
  'Lead Generation',
  'Social Media',
  'Public Relations',
  'Product Marketing',
  'Marketing Management',
  'Marketing Strategy',
  'Vendor Management Experience'],
 'industry': 'Marketing and Advertising',
 'summary': 'MARKETING MANAGERDynamic and confident marketing professional with extensive skills needed to support business, consumer, and product market programs. Detail-oriented individual recognized for pro-active approach and an effective team player to tactical plans with a strong desire to meet organizational goals in order to make a positive contribution.QUALIFICATIONS SUMMARY:• Improved cost efficiency by 15% through successful asset inventory management, vendor rela

## Designing a relational schema from hierarchical data

Given that we already have a predefined set of fields / attributes / features, we don't need to spend a lot of time defining our table *schemas*, except that we need to unnest data.

* Nested relationships can be captured by creating a second table, which has a **foreign key** pointing to the identifier (key) for the main (parent) table.
* Ordered lists can be captured by encoding an index number or row number.

In [13]:
'''
Simple code to pull out data from JSON and load into DuckDB.
'''
import ast

linked_in = open('linkedin_anon.jsonl')

START = 0
LIMIT = 10000

def get_df(rel):
    ret = pd.DataFrame(rel)
    return ret

lines = []
i = 1
for line in linked_in:
    if i > START + LIMIT:
        break
    elif i >= START:
        person = json.loads(line)

        lines.append(person)
    i = i + 1

people_df = get_df(pd.DataFrame(lines))



In [14]:
people_df

,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,moist-vodka,"{'family_name': 'Post', 'given_name': 'Belvede...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,https://www.linkedin.com/in/moist-vodka,None,None,NaN,None,None,None,None,None
1,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
2,tart-acorn,"{'family_name': 'Hannay', 'given_name': 'Passe...","San Francisco, California","[DNA, Nanotechnology, Molecular Biology, Softw...",Research,I am interested in inventing new methods to co...,https://www.linkedin.com/in/tart-acorn,"[{'major': 'Biophysics', 'end': '2009', 'name'...",None,0.0,"[{'org': 'UCSF', 'title': 'Assistant Professor...",None,[{'from': 'Wyss Institute for Biologically Ins...,"personal genomics, nanotechnology",None
3,objective-riesling,"{'family_name': 'Carnegie', 'given_name': 'Pas...",San Francisco Bay Area,None,Information Technology and Services,OBJECTIVE<Primary> Work on an interesting and ...,https://www.linkedin.com/in/objective-riesling,None,"{'affilition': ['Big Data, Low Latency', 'Expe...",5.0,"[{'org': '<Online Recruiting Company>', 'desc'...",None,"[{'from': '<Employee Benefits, Administration ...",None,None
4,generative-amberjack,"{'family_name': 'Duncan', 'given_name': 'Merri...","Chennai Area, India","[Program Management, French, Avionics, Embedde...",Aviation & Aerospace,"Experience in Avionics Systems, Embedded Syste...",https://www.linkedin.com/in/generative-amberjack,"[{'start': '1988', 'end': '1989', 'name': 'Eco...",{'member': 'Member of Project Management Insti...,NaN,None,None,None,"Literature, Philosophy, Music",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,gourmet-block,"{'family_name': 'Drummond', 'given_name': 'Alf...",Singapore,None,Chemicals,Site manager for a chemical MC with extensive ...,https://www.linkedin.com/in/gourmet-block,"[{'major': 'Applied Finance', 'end': '2007', '...","{'affilition': ['COMPANY PHARMA TALENT', 'Chem...",22.0,"[{'org': 'Perstorp Singapore Pte Ltd', 'title'...","Operations, Production chemical petrochemical,...",[{'from': 'Stazione Sperimentale per I Combust...,None,None
9996,matte-curve,"{'family_name': 'Rollo', 'given_name': 'Bullim...","Sevilla y alrededores, España","[Automotive, Marketing Strategy, Product Manag...",Sector automovilístico,None,https://www.linkedin.com/in/matte-curve,[{'name': 'Licenciado en Marketing e investiga...,None,9.0,"[{'org': 'Glassdrive España', 'title': 'Direct...",None,"[{'from': 'Saint-Gobain Glassdrive España', 't...",None,None
9997,adiabatic-pilot,"{'family_name': 'Barclay', 'given_name': 'Simo...","Gijón y alrededores, España","[Intranet, Spanish, Personnel Management, Inte...",Minería y metalurgia,"In my current position, I've faced two key cha...",https://www.linkedin.com/in/adiabatic-pilot,"[{'major': 'HR Management', 'end': '2001', 'na...","{'affilition': ['ArcelorMittal Group', 'Basket...",19.0,"[{'org': 'ArcelorMittal', 'title': 'Head of In...",None,"[{'from': 'Secades, Lozano y Tejon', 'to': 'Pr...",None,None
9998,advanced-object,"{'family_name': 'Hope', 'given_name': 'Jeeves'}","Nice Area, France","[Amadeus, Project Coordination, Project Manage...",Technologies et services de l'information,None,https://www.linkedin.com/in/advanced-object,[{'major': 'Computer science and telecommunica...,None,0.0,"[{'org': 'Amadeus IT Group', 'title': 'Impleme...",None,"[{'from': 'Vodafone IT

In [15]:
def get_nested_dict(rel, name):
  # This evaluates the string that describes the dictionary, as a dictionary
  # definition
  ret = rel.copy()
  # ret[name] = rel[name].map(lambda x: ast.literal_eval(x) if len(x) else np.NaN)
  ret = ret.dropna()
  # This joins rows on the index
  return ret.drop(columns=name).join(pd.DataFrame(ret[name].tolist()))

def get_nested_list(rel, name):
  ret = rel.copy()
  ret = ret.dropna().explode(name).dropna()
  ret = ret.join(pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()
  return ret.rename(columns={0: name})

def get_nested_list_dict(rel, name):
  ret = rel.copy()

  ret = ret.dropna().explode(name)

  exploded_pairs = pd.DataFrame(ret.apply(lambda x: {'_id': x['_id']} | x[name] if isinstance(x[name], dict) else {'_id': x['_id']}, axis=1).tolist())

  return ret.merge(exploded_pairs, on='_id').drop(columns=name)
  #pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()

# Take the lists, drop any blank strings
specialties_df = people_df[['_id','specilities']].explode('specilities').rename(columns={'_id': 'person'})
specialties_df.dropna(inplace=True)
interests_df = people_df[['_id','interests']].explode('interests').rename(columns={'_id': 'person'})
interests_df.dropna(inplace=True)

names_df = get_nested_dict(people_df[['_id','name']], 'name')

education_df = get_nested_list_dict(people_df[['_id','education']], 'education')
experience_df = get_nested_list_dict(people_df[['_id','experience']], 'experience')
skills_df = get_nested_list(people_df[['_id','skills']], 'skills')
honors_df = get_nested_list(people_df[['_id','honors']], 'honors')
events_df = get_nested_list_dict(people_df[['_id','events']], 'events')

groups_df = get_nested_dict(people_df[['_id','group']], 'group')

people_df = people_df.drop(columns=['name','education','group','skills','experience','honors','events','specilities','interests'])


In [16]:
events_df

,_id,from,to,title1,start,title2,end
0,adagio-catalyst,Sahlgrenska University Hospital,Memorial Sloan Kettering Cancer Center,Research Scientist,24022.0,Post Doctoral Research Fellow,24036.0
1,adagio-catalyst,Memorial Sloan Kettering Cancer Center,Columbia University,Post Doctoral Research Fellow,24036.0,Associate Research Scientist,24079.0
2,adagio-catalyst,Columbia University,Albert Einstein Medical Center,Associate Research Scientist,24079.0,Associate at Dept of Molecular Genetics,24104.0
3,adagio-catalyst,Albert Einstein Medical Center,Johnson and Johnson,Associate at Dept of Molecular Genetics,24104.0,"Senior Scientist, Oncology Biomarkers",24118.0
4,adagio-catalyst,Sahlgrenska University Hospital,Memorial Sloan Kettering Cancer Center,Research Scientist,24022.0,Post Doctoral Research Fellow,24036.0
...,...,...,...,...,...,...,...
229237,advanced-object,Amadeus,Amadeus IT Group,Product definition engineer,24102.0,Implementation engineer,24121.0
229238,inflammable-tarragon,Koch Media srl,Atari Games,Sales Manager Italy,24039.0,sales director,24070.0
229239,inflammable-tarragon,Atari Games,Namco Bandai Partners,sales director,24070.0,Sales Director,24114.0
229240,inflammable-tarragon,Koch Media srl,Atari Games,Sales Manager Italy,24039.0,sales director,24070.0


In [17]:
interests_df

,person,interests
2,tart-acorn,"personal genomics, nanotechnology"
4,generative-amberjack,"Literature, Philosophy, Music"
5,salty-section,"travelling,the sea,trying new things, trying t..."
9,rich-laser,Marketing and statistical marketing applicatio...
10,cold-reveal,"Fashion Photography, Public Relations, Marketi..."
...,...,...
9980,delicious-voxel,Programming and Clean Code
9984,cerulean-gabardine,"free software, copyleft, open source, linux, p..."
9987,linear-foie-gras,"Keen on motorbike travelling, sailing, diving ..."
9990,radiant-rottweiler,"Arte, Diseño, 3D, Programación, Flash, Motion ..."


In [18]:
specialties_df

,person,specilities
1,adagio-catalyst,"Biomarkers in Oncology, Cancer Genomics, Molec..."
5,salty-section,"A passion for Brands, coupled with experience ..."
13,forgiving-desert,"Internet Marketing, Interactive Marketing, Dig..."
16,plain-torpedo,"Marketing , Operations Management , P&L Head, ..."
23,cheerful-mackerel,"SQL, DB2, COBOL,JCL, JAVA"
...,...,...
9984,cerulean-gabardine,"Web Design, Web Development, Web Standards, We..."
9988,camel-mason,"Innovation, comunication, social media"
9992,callous-horror,"Marketing, Advertising, Commercial & Business ..."
9995,gourmet-block,"Operations, Production chemical petrochemical,..."


In [19]:
names_df

,_id,family_name,given_name
0,moist-vodka,Post,Belvedere
1,adagio-catalyst,Watt,Brunton
2,tart-acorn,Hannay,Passepartout
3,objective-riesling,Carnegie,Passepartout
4,generative-amberjack,Duncan,Merriman
...,...,...,...
9995,gourmet-block,Drummond,Alfred
9996,matte-curve,Rollo,Bullimore
9997,adiabatic-pilot,Barclay,Simonides
9998,advanced-object,Hope,Jeeves


In [20]:
education_df

,_id,start,major,end,name,desc,degree
0,adagio-catalyst,2008,Economics,2008,Columbia University - Columbia Business School,"Coursework ""Principals of Economics"" ECON1105\...",NaN
1,adagio-catalyst,2007,NaN,2007,Columbia University - Columbia Business School,NaN,NaN
2,adagio-catalyst,1996,Cancer genomics,2001,Göteborgs universitet,"Thesis: ""The role of p53 in tumor progression ...",Ph.D.
3,adagio-catalyst,1994,"Biology, Medicine;German Language",1995,Universität Regensburg,NaN,"Cancer Research, Coursework"
4,adagio-catalyst,1989,Biology,1994,Göteborgs universitet,,Master
...,...,...,...,...,...,...,...
60872,advanced-object,2005,Computer science and telecommunications,2008,Politecnico di Torino,"Degree thesis: ""Vehicle-ground communication b...",Telematics engineer
60873,advanced-object,2002,Computer science,2005,Politecnico di Torino,,Computer engineer
60874,advanced-object,2005,Computer science and telecommunications,2008,Politecnico di Torino,"Degree thesis: ""Vehicle-ground communication b...",Telematics engineer
60875,advanced-object,2002,Computer science,2005,Politecnico di Torino,,Computer engineer


In [21]:
experience_df

,_id,org,title,end,start,desc
0,adagio-catalyst,Johnson and Johnson,"Senior Scientist, Oncology Biomarkers",Present,November 2009,Biomarker Leader for compounds in clinical dev...
1,adagio-catalyst,Albert Einstein Medical Center,Associate at Dept of Molecular Genetics,NaN,September 2008,Single Cell Gene expression.
2,adagio-catalyst,Columbia University,Associate Research Scientist,NaN,August 2006,Work on peptide to restore wt p53 function in ...
3,adagio-catalyst,Memorial Sloan Kettering Cancer Center,Post Doctoral Research Fellow,NaN,January 2003,Molecular profiling of colorectal cancer.
4,adagio-catalyst,Sahlgrenska University Hospital,Research Scientist,NaN,November 2001,Cancer Research at Dept of Surgery.Molecular p...
...,...,...,...,...,...,...
339192,inflammable-tarragon,Atari Games,sales director,NaN,November 2005,"Sales Director Italiy , for a Branch of Atari ..."
339193,inflammable-tarragon,Koch Media srl,Sales Manager Italy,NaN,April 2003,Sales Manager Italy distributor of software an...
339194,inflammable-tarragon,Namco Bandai Partners,Sales Director,Present,July 2009,NaN
339195,inflammable-tarragon,Atari Games,sales director,NaN,November 2005,"Sales Director Italiy , for a Branch of Atari ..."


In [22]:
groups_df


,_id,affilition,member
1,adagio-catalyst,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
3,objective-riesling,"[Canadian Marketing Association, LeadingLoyalt...",NaN
4,generative-amberjack,"[CFA Institute Candidates, Economist Intellige...",Associate Member of SAMRA
6,chalky-tenement,"[BMW Group, BPO Executives, Engineering jobs B...",NaN
9,rich-laser,"[Annamalai University Alumni, Annamalai Univer...",NaN
...,...,...,...
9992,callous-horror,NaN,NaN
9993,equidistant-lumen,NaN,NaN
9994,savory-dimension,NaN,NaN
9995,gourmet-block,NaN,NaN


In [23]:
conn = duckdb.connect('linkedin.db')

conn.sql('drop table if exists people')
conn.sql('drop table if exists names')
conn.sql('drop table if exists education')
conn.sql('drop table if exists groups')
conn.sql('drop table if exists skills')
conn.sql('drop table if exists experience')
conn.sql('drop table if exists honors')
conn.sql('drop table if exists events')
conn.sql('drop table if exists specialties')
conn.sql('drop table if exists interests')

In [24]:
# Save these to the SQLite database

conn.sql("""
  CREATE TABLE IF NOT EXISTS people AS
   SELECT * FROM people_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS names AS
   SELECT * FROM names_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS education AS
   SELECT * FROM education_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS groups AS
   SELECT * FROM groups_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS skills AS
   SELECT * FROM skills_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS experience AS
   SELECT * FROM experience_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS honors AS
   SELECT * FROM honors_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS events AS
   SELECT * FROM events_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS specialties AS
   SELECT * FROM specialties_df
""")

conn.sql("""
  CREATE TABLE IF NOT EXISTS interests AS
   SELECT * FROM interests_df
""")

In [25]:
conn.sql("""
  SELECT experience._id, org
  FROM people
  JOIN experience ON people._id=experience._id""")

┌─────────────────────┬─────────────────────────┐
│         _id         │           org           │
│       varchar       │         varchar         │
├─────────────────────┼─────────────────────────┤
│ collinear-monastery │ SimTel                  │
│ collinear-monastery │ Radmor S.A              │
│ collinear-monastery │ Kainos                  │
│ collinear-monastery │ Misys                   │
│ collinear-monastery │ Nordea Bank Polska S.A. │
│ collinear-monastery │ DGT                     │
│ collinear-monastery │ GE Money Bank           │
│ collinear-monastery │ GE Money Bank           │
│ collinear-monastery │ SimTel                  │
│ collinear-monastery │ Radmor S.A              │
│    ·                │    ·                    │
│    ·                │    ·                    │
│    ·                │    ·                    │
│ glum-bag            │ Ericsson                │
│ glum-bag            │ Ericsson                │
│ glum-bag            │ Cisco                   │


In [26]:
conn.sql("""
  SELECT experience._id, group_concat(org) AS experience
  FROM people
  LEFT JOIN experience ON people._id=experience._id
  GROUP BY experience._id""")

┌──────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Views

The following code starts a transaction (we can either `commit` or `rollback` at the end), removes an existing view, and creates a new one.

In [27]:
conn.sql('BEGIN TRANSACTION')
conn.sql('DROP VIEW IF EXISTS people_experience')
conn.execute("""
  CREATE VIEW IF NOT EXISTS people_experience AS
    SELECT experience._id, group_concat(org) AS experience
    FROM people
    LEFT JOIN experience ON people._id=experience._id
    GROUP BY experience._id""")
conn.execute('COMMIT')

# Treat the view as a table, see what's there
conn.sql('SELECT * FROM people_experience')

┌──────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Deep Dive: Converting a Complex Tree to Relations

Now that we've seen the basics of taking hierarchical data and turning it into relations, let's put the LinkedIn data on the stack for a brief time, and try a more difficult exercise representing (and querying) tree-structured data.

We'll take the HTML data from Wikipedia pages, seen in the Lecture 1 Notebook, and "shred" the HTML into tables.

Briefly, if we think of the HTML as a tree of nodes, e.g.:

```
   <html>
   |   |
<head> <body>
   |    |   |
<title> <h1> <p>
   |     |    \
 ABC    ABC    DEF
```

Then we can give a **node ID** to each node in the tree; an a **position** (0, 1, ...) to each sibling at a level in the tree.  We will "slice" the tree into segments, each of which becomes a row in a table.  The row will include the node ID, the node label or type ("h1" or "text()"), the node value if the type is text(), and the position.

In [28]:
import urllib
from lxml import etree
import pandas as pd
import requests


## HTML as edges

Each time we parse an HTML node, we can give it a new ID.  If we record the ID of its parent, we essentially get an _edge_ going back to the parent.

In [29]:
def import_html(url: str):
  # Now let's read an HTML table!
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }

  return requests.get(url, headers=headers).text


# Recursively crawl the node and add rows to the html_tree table
def traverse_html(node, parent, pos, nodes_list) -> list:
    if node.text and parent > -1 and len(str(node.text).strip()):
        text_id = len(nodes_list)
        entry = {'node_id': text_id, 'parent_node_id': parent, 'type_or_label': 'text()', 'pos': pos, 'value': str(node.text).strip()}
        print (str(entry))
        nodes_list.append(entry)

    if node.tag:
        node_id = len(nodes_list)
        entry = {'node_id': node_id, 'parent_node_id': parent, 'type_or_label': node.tag, 'pos': pos, 'value': ''}
        nodes_list.append(entry)
        print (str(entry))
        index = 0
        for child in list(node):
            (child_id, nodes_list) = traverse_html(child, node_id, index, nodes_list)
            index = index + 1

    if node.tail:
        text_id = len(nodes_list)
        entry = {'node_id': text_id, 'parent_node_id': parent, 'type_or_label': 'text()', 'pos': pos, 'value': node.tail}
        print (str(entry))
        nodes_list.append(entry)
    return (node_id, nodes_list)

pages_list = []
nodes_list = []


# Crawl these pages
page_list = ['https://en.wikipedia.org/wiki/Tim_Cook',
             'https://en.wikipedia.org/wiki/Chan_Zuckerberg_Initiative']
for page in page_list:
    page_content = import_html(page)
    page_tree = etree.HTML(page_content)
    (root_node,nodes_list) = traverse_html(page_tree, -1, 0, nodes_list)
    pages_list.append({'url': page, 'root_id': root_node})

pages_df = pd.DataFrame(pages_list)
pages_df

Streaming output truncated to the last 5000 lines.
{'node_id': 8735, 'parent_node_id': 8697, 'type_or_label': 'meta', 'pos': 17, 'value': ''}
{'node_id': 8736, 'parent_node_id': 8697, 'type_or_label': 'text()', 'pos': 17, 'value': '\n'}
{'node_id': 8737, 'parent_node_id': 8697, 'type_or_label': 'meta', 'pos': 18, 'value': ''}
{'node_id': 8738, 'parent_node_id': 8697, 'type_or_label': 'text()', 'pos': 18, 'value': '\n'}
{'node_id': 8739, 'parent_node_id': 8697, 'type_or_label': 'link', 'pos': 19, 'value': ''}
{'node_id': 8740, 'parent_node_id': 8697, 'type_or_label': 'text()', 'pos': 19, 'value': '\n'}
{'node_id': 8741, 'parent_node_id': 8697, 'type_or_label': 'link', 'pos': 20, 'value': ''}
{'node_id': 8742, 'parent_node_id': 8697, 'type_or_label': 'text()', 'pos': 20, 'value': '\n'}
{'node_id': 8743, 'parent_node_id': 8697, 'type_or_label': 'link', 'pos': 21, 'value': ''}
{'node_id': 8744, 'parent_node_id': 8697, 'type_or_label': 'text()', 'pos': 21, 'value': '\n'}
{'node_id': 8745, '

,url,root_id
0,https://en.wikipedia.org/wiki/Tim_Cook,0
1,https://en.wikipedia.org/wiki/Chan_Zuckerberg_...,8696


Let's see the nodes and the edges to their parents.  A parent of `-1` is a root node.

In [30]:
node_df = pd.DataFrame(nodes_list)
node_df

,node_id,parent_node_id,type_or_label,pos,value
0,0,-1,html,0,
1,1,0,head,0,
2,2,1,meta,0,
3,3,1,text(),0,\n
4,4,1,text(),1,Tim Cook - Wikipedia
...,...,...,...,...,...
13730,13730,8764,text(),5,\n
13731,13731,8764,text(),6,"{""@context"":""https:\/\/schema.org"",""@type"":""Ar..."
13732,13732,8764,script,6,
13733,13733,8764,text(),6,\n


Let's look at the pages and the IDs of their root nodes.

In [31]:
pages_df

,url,root_id
0,https://en.wikipedia.org/wiki/Tim_Cook,0
1,https://en.wikipedia.org/wiki/Chan_Zuckerberg_...,8696


From the pages, we can join the root nodes and see what the tags are.

In [32]:
# Find all document roots
pages_df.merge(node_df,left_on=['root_id'],right_on=['node_id'])

,url,root_id,node_id,parent_node_id,type_or_label,pos,value
0,https://en.wikipedia.org/wiki/Tim_Cook,0,0,-1,html,0,
1,https://en.wikipedia.org/wiki/Chan_Zuckerberg_...,8696,8696,-1,html,0,


Now let's consider an XPath query to find all text within paragraphs.

This would be `//p/text()`. We can evaluate this easily by just looking for `p` elements whose children are text. This can be done by joining between nodes with `p`s and nodes with text.

In [33]:
# Return the contents of all text() nodes inside of <p> tags

node_df[node_df['type_or_label']=='p'][['node_id']].\
    merge(node_df[node_df['type_or_label']=='text()'], \
          left_on=['node_id'], right_on=['parent_node_id'])[['value']]

,value
0,Timothy Donald Cook
1,"(born November 1, 1960)"
2,is an American business executive who is the ...
3,Apple Inc.
4,Cook had previously been the company's chief ...
...,...
474,and has fewer other transparency requirements...
475,"Under this legal structure,"
476,"wrote, ""Zuckerberg will still control the Fac..."
477,The Chan Zuckerberg Initiative publicly lists...


It's potentially more informative to see a bit of context.  Let's show (1) the node ID of the parent of the `p` tag, (2) the node ID of the `p` tag, (3) the node ID of the text node.

In [34]:
p_text_nodes = node_df[node_df['type_or_label']=='p'][['parent_node_id','node_id']].\
    merge(node_df[node_df['type_or_label']=='text()'][['parent_node_id','node_id']], \
          left_on=['node_id'], right_on=['parent_node_id']).\
    rename(columns={'parent_node_id_x': 'p_parent_node_id', 'node_id_y': 'text_node_id'}).\
    drop(columns='node_id_x').rename(columns={'parent_node_id_y': 'p_node_id'})

p_text_nodes

,p_parent_node_id,p_node_id,text_node_id
0,1108,1216,1217
1,1108,1216,1219
2,1108,1216,1227
3,1108,1216,1228
4,1108,1216,1230
...,...,...,...
474,9562,10580,10649
475,9562,10580,10678
476,9562,10580,10682
477,9562,10580,10697


What can we say about the types of the parent's of the `p`-nodes?

In [35]:
current_items_df = p_text_nodes.rename(columns={'p_parent_node_id': 'ancestor_node_id'})

parents_df = current_items_df[['ancestor_node_id','text_node_id']].\
    merge(node_df,\
    left_on=['ancestor_node_id'],right_on=['node_id'])\
    [['parent_node_id','text_node_id','type_or_label']].\
rename(columns={'parent_node_id': 'ancestor_node_id'})

parents_df

,ancestor_node_id,text_node_id,type_or_label
0,1107,1217,div
1,1107,1219,div
2,1107,1227,div
3,1107,1228,div
4,1107,1230,div
...,...,...,...
474,9561,10649,div
475,9561,10678,div
476,9561,10682,div
477,9561,10697,div


And we can even traverse once more, to the parents of the parents!

In [36]:
current_items_df = parents_df

grandparents_df = current_items_df[['ancestor_node_id','text_node_id']].drop_duplicates().\
    merge(node_df,\
    left_on=['ancestor_node_id'],right_on=['node_id'])\
    [['parent_node_id','text_node_id','type_or_label']].\
rename(columns={'parent_node_id': 'ancestor_node_id'}).drop_duplicates()

grandparents_df

,ancestor_node_id,text_node_id,type_or_label
0,1090,1217,div
1,1090,1219,div
2,1090,1227,div
3,1090,1228,div
4,1090,1230,div
...,...,...,...
474,9550,10649,div
475,9550,10678,div
476,9550,10682,div
477,9550,10697,div


## Recursively find all ancestors!

We can start with the text nodes, then find their parents, then find their parents, then ...

This is a recursive process that stops when there aren't any more parents, and is called a *transitive closure* because it includes the full set of all transitively related nodes.

In [37]:
def find_ancestor_nodes(node_df, current_items_df):
    if len(current_items_df) == 0:
        return current_items_df
    else:
        parents_df = current_items_df[['ancestor_node_id','text_node_id']].drop_duplicates().\
            merge(node_df,\
            left_on=['ancestor_node_id'],right_on=['node_id'])\
            [['parent_node_id','text_node_id','type_or_label']].\
        rename(columns={'parent_node_id': 'ancestor_node_id'}).drop_duplicates()

        return pd.concat([parents_df,find_ancestor_nodes(node_df, parents_df)]).drop_duplicates()

nodes_ancestors = find_ancestor_nodes(node_df, p_text_nodes.rename(columns={'p_parent_node_id': 'ancestor_node_id'}))

nodes_ancestors

,ancestor_node_id,text_node_id,type_or_label
0,1107,1217,div
1,1107,1219,div
2,1107,1227,div
3,1107,1228,div
4,1107,1230,div
...,...,...,...
474,-1,10649,html
475,-1,10678,html
476,-1,10682,html
477,-1,10697,html


In [38]:
# Can we find ONLY text from the Tim Cook (0th) document?

nodes_ancestors[nodes_ancestors['ancestor_node_id']==pages_df.iloc[0]['root_id']].\
    merge(node_df, left_on=['text_node_id'],right_on=['node_id'])[['text_node_id','value']]

,text_node_id,value
0,1217,Timothy Donald Cook
1,1219,"(born November 1, 1960)"
2,1227,is an American business executive who is the ...
3,1228,Apple Inc.
4,1230,Cook had previously been the company's chief ...
...,...,...
291,2537,\n
292,5688,"""Tim Cook, Making Apple His Own"""
293,5690,.
294,5691,The New York Times


## Exercises

In [39]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [40]:
!pip3 install penngrader-client

In [41]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 51778640 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [42]:
%set_env HW_ID=cis5450_25f_HW9

env: HW_ID=cis5450_25f_HW9


In [43]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 51778640

Make sure this correct or we will not be able to store your grade




Can we find *all paragraphs* from the Zuckerberg (position-1) document, that have text children? (Hint: you can get all text nodes in the document, then find their parents). We'll give you a starting point here:

In [44]:
zuckerberg_root_id = pages_df.iloc[1]['root_id']
zuckerberg_root_id

np.int64(8696)

In [45]:
import pandas as pd
node_df = pd.DataFrame(nodes_list)
node_df

,node_id,parent_node_id,type_or_label,pos,value
0,0,-1,html,0,
1,1,0,head,0,
2,2,1,meta,0,
3,3,1,text(),0,\n
4,4,1,text(),1,Tim Cook - Wikipedia
...,...,...,...,...,...
13730,13730,8764,text(),5,\n
13731,13731,8764,text(),6,"{""@context"":""https:\/\/schema.org"",""@type"":""Ar..."
13732,13732,8764,script,6,
13733,13733,8764,text(),6,\n


In [55]:
# TODO: Return (node_id, type_or_label) as results_df

text_with_parents = node_df[node_df['type_or_label'] == 'text()'][['node_id','parent_node_id']].rename(columns={'node_id':'text_node_id','parent_node_id':'ancestor_node_id'})
all_ancestors = find_ancestor_nodes(node_df,text_with_parents)
zuckerberg_text_ids = all_ancestors[all_ancestors['ancestor_node_id'] == zuckerberg_root_id][['text_node_id']].drop_duplicates()
parents_of_text = zuckerberg_text_ids.merge(node_df[['node_id','parent_node_id']],left_on = 'text_node_id',right_on = 'node_id',how='left').rename(columns={'parent_node_id':'parent_node_id_of_text'})
paragraph_nodes = parents_of_text.merge(node_df[['node_id','type_or_label']],left_on='parent_node_id_of_text',right_on='node_id',how='left',suffixes=('_text', '_parent'))

results_df = paragraph_nodes[paragraph_nodes['type_or_label']=='p'][['node_id_parent','type_or_label']].drop_duplicates().rename(columns={'node_id_parent':'node_id'})

results_df

,node_id,type_or_label
379,9765,p
399,9837,p
401,9855,p
404,9868,p
410,9892,p
413,9929,p
418,9945,p
433,10042,p
444,10074,p
453,10096,p


Quick-check for your columns.

In [56]:
assert list(results_df.columns)==['node_id', 'type_or_label']

And submit!

In [57]:
grader.grade('zuckerberg', results_df)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
